In [56]:
import numpy as np
import pandas as pd
import requests
import sqlite3
import warnings
from datetime import datetime
from bs4 import BeautifulSoup
warnings.filterwarnings('ignore')

In [57]:
url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
ex_rate_csv_path = "exchange_rate.csv"
output_path = "Largest_banks_data.csv"
database = "Banks.db"
conn = sqlite3.connect(database)

In [58]:
# EXTRACT

def extract(url):
    ''' This function aims to extract the required information from the website and save it to a data frame. The function returns the data frame for further processing. '''
    res = requests.get(url)
    print("Status:", str(res.status_code))
    html_page = res.text
    soup = BeautifulSoup(html_page, 'html.parser')

    table_body = soup.findAll('tbody')[0]
    cols = []
    for col in table_body.findAll('th'):
        cols.append(col.text.replace(' ', '_').replace('\n', '').replace('$', 'D'))

    df = pd.DataFrame(columns=cols)

    for rows in table_body.findAll('tr'):
        if len(rows.findAll('td')) > 0:
            temp_dict = {cols[0]: rows.findAll('td')[0].text.replace('\n', ''),
                        cols[1]: rows.findAll('td')[1].text.replace('\n', ''),
                        cols[2]: float(rows.findAll('td')[2].text.replace('\n', ''))}
            temp_df = pd.DataFrame(temp_dict, index=[0])
            df = pd.concat([df, temp_df], ignore_index=True)

    msg = "Extract phase ended"
    return df, msg

In [59]:
# TRANSFORM

def transform(df, ex_rate_csv_path):
    ''' This function accesses the CSV file for exchange rate information, and adds three columns to the data frame, each containing the transformed version of Market Cap column to respective currencies'''
    ex_rate = pd.read_csv(ex_rate_csv_path, header=0)

    inr = ex_rate[ex_rate['Currency'] == "INR"]['Rate']
    gbp = ex_rate[ex_rate['Currency'] == "GBP"]['Rate']
    eur = ex_rate[ex_rate['Currency'] == "EUR"]['Rate']

    df['MC_INR_Billions'] = np.nan
    df['MC_GBP_Billions'] = np.nan
    df['MC_EUR_Billions'] = np.nan

    # for inr
    for i in range(len(df)):
        df['MC_INR_Billions'].iloc[i] = round(df['Market_cap(USD_billion)'].iloc[i] * inr, 2)

    # for gbp
    for j in range(len(df)):
        df['MC_GBP_Billions'].iloc[j] = round(df['Market_cap(USD_billion)'].iloc[j] * gbp, 2)
    
    # for eur
    for k in range(len(df)):
        df['MC_EUR_Billions'].iloc[k] = round(df['Market_cap(USD_billion)'].iloc[k] * eur, 2)

    msg = "Transform phase ended"
    return df, msg

In [60]:
# LOAD
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in the provided path. Function returns nothing.'''
    df.to_csv(output_path)
    msg = "Loading to csv, ended"
    return msg

def load_to_db(df, conn):
    ''' This function saves the final data frame to a database table with the provided name. Function returns nothing.'''
    df.to_sql("Largest_banks", con=conn, if_exists="replace", index=True, index_label="Id")
    msg = "Loading to sql, ended"
    # conn.close()
    return msg

In [61]:
# LOG
def log_progress(msg):
    ''' This function logs the mentioned message of a given stage of the code execution to a log file. Function returns nothing'''
    now = datetime.now() # current date and time
    timestamp_format = now.strftime("%Y/%m/%d-%H:%M:%S")
    ts = timestamp_format

    with open("Log.txt", "a") as f:
        f.write(f'{ts}: {msg}')
        f.write('\n')

In [62]:
def run_query(statement, conn):
    '''This function runs the query on the database table and prints the output on the terminal. Function returns nothing.'''
    try:
        result = pd.read_sql(f'{statement}', con=conn)
        msg = 'Query executed successfully'
    except:
        msg = 'Query unsuccessful!'
    return result, msg

In [63]:
msg = "Extract phase Started"
log_progress(msg)
df, Emsg = extract(url)
log_progress(Emsg)

msg = "Transform phase Started"
log_progress(msg)
df, Tmsg = transform(df, ex_rate_csv_path)
log_progress(Tmsg)

msg = "Loading to csv, Started"
log_progress(msg)
csv_msg = load_to_csv(df, output_path)
log_progress(csv_msg)

msg = "Loading to sql, Started"
log_progress(msg)
sql_msg = load_to_db(df, conn)
log_progress(sql_msg)

Status: 200


In [64]:
msg = "Running Query"
log_progress(msg)
statement = 'SELECT * FROM Largest_banks'
rslt, Qmsg = run_query(statement, conn)
log_progress(Qmsg)
rslt

,Id,Rank,Bank_name,Market_cap(USD_billion),MC_INR_Billions,MC_GBP_Billions,MC_EUR_Billions
0,0,1,JPMorgan Chase,432.92,35910.71,346.34,402.62
1,1,2,Bank of America,231.52,19204.58,185.22,215.31
2,2,3,Industrial and Commercial Bank of China,194.56,16138.75,155.65,180.94
3,3,4,Agricultural Bank of China,160.68,13328.41,128.54,149.43
4,4,5,HDFC Bank,157.91,13098.63,126.33,146.86
5,5,6,Wells Fargo,155.87,12929.42,124.70,144.96
6,6,7,HSBC Holdings PLC,148.90,12351.26,119.12,138.48
7,7,8,Morgan Stanley,140.83,11681.85,112.66,130.97
8,8,9,China Construction Bank,139.82,11598.07,111.86,130.03
9,9,10,Bank of China,136.81,11348.39,109.45,127.23


In [65]:
msg = "Running Query"
log_progress(msg)
statement = 'SELECT AVG(MC_GBP_Billions) FROM Largest_banks'
rslt, Qmsg = run_query(statement, conn)
log_progress(Qmsg)
rslt

,AVG(MC_GBP_Billions)
0,151.987


In [66]:
msg = "Running Query"
log_progress(msg)
statement = 'SELECT Bank_name from Largest_banks LIMIT 5'
rslt, Qmsg = run_query(statement, conn)
log_progress(Qmsg)
rslt

,Bank_name
0,JPMorgan Chase
1,Bank of America
2,Industrial and Commercial Bank of China
3,Agricultural Bank of China
4,HDFC Bank


In [67]:
# Closing DB connection
conn.close()